In [ ]:
import os, sys
import pandas as pd
import json

sys.path.append(os.path.join(os.getcwd(), '../src'))
import faissEncoder as faiss_enc
from crossEncoder import CrossEncoderReranker
from metrics import calculate_topk_accuracy
from utils import load_corpus_data, evaluate_model


/home/fernandogd/Documents/Investigacion/Transformers/Repositories/ClinLinker-KB/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
CORPORA = ["DisTEMIST", "MedProcNER", "SympTEMIST"]
model_map = {
    "SapBERT-UMLS-2020AB-all-lang-from-XLMR-large" : "SapBERT-XLM-R-large",
    "SapBERT-UMLS-2020AB-all-lang-from-XLMR" : "SapBERT-XLM-R-base",
    "sapbert_15_noparents_1epoch" : "ClinLinker",
    "sapbert_15_parents_1epoch" : "ClinLinker-KB-P",
    "sapbert_15_grandparents_1epoch" : "ClinLinker-KB-GP",
    "distemist-biencoder" : "DisTEMIST-bi-encoder",
    "medprocner-biencoder" : "MedProcNER-bi-encoder",
    "symptemist-biencoder" : "SympTEMIST-bi-encoder",
    "roberta-base-biomedical-clinical-es": "RoBERTa-base-biomedical"
}

F_TYPE = "FlatIP"
MAX_LENGTH = 256

DATA_PATH = "/scratch/data/"
TOP_K_VALUES = [1, 5, 25, 50, 100, 200]


In [ ]:
for corpus in CORPORA:
    gs_results, clean_results, um_results, uc_results = dict(), dict(), dict(), dict()
    um_df = pd.read_csv(f"../data/{corpus}/df_um.tsv", sep="\t", dtype={"code":str})
    uc_df = pd.read_csv(f"../data/{corpus}/df_uc.tsv", sep="\t", dtype={"code":str})
    gs_df, train_df, gaz_df = load_corpus_data(DATA_PATH, corpus)
    train_gaz_df = pd.concat([train_df[["term", "code"]], gaz_df[["term","code"]]], ignore_index=True)
    clean_df = gs_df[
        gs_df['code'].notna() & 
        (gs_df['code'] != "NO_CODE") & 
        (~gs_df['code'].str.contains("\+", na=False))
    ]

    model_list = [
        "/scratch/models/NEL/spanish_sapbert_models/sapbert_15_grandparents_1epoch",
        "/scratch/models/NEL/spanish_sapbert_models/sapbert_15_parents_1epoch",
        "/scratch/models/NEL/spanish_sapbert_models/sapbert_15_noparents_1epoch",
        f"/scratch/models/NEL/corpus-specific_bi-encoders/{corpus.lower()}-biencoder",
        "cambridgeltl/SapBERT-UMLS-2020AB-all-lang-from-XLMR-large",
        "cambridgeltl/SapBERT-UMLS-2020AB-all-lang-from-XLMR",
        "PlanTL-GOB-ES/roberta-base-biomedical-clinical-es"
    ]
    for model in model_list:
        gs_results, clean_results, um_results, uc_results=evaluate_model(
            model_name=model,
            f_type=F_TYPE,
            max_length=MAX_LENGTH,
            train_gaz_df=train_gaz_df,
            gs_df=gs_df,
            clean_df=clean_df,
            um_df=um_df,
            uc_df=uc_df,
            gs_results=gs_results,
            clean_results=clean_results,
            um_results=um_results,
            uc_results=uc_results,
            model_map=model_map,
            top_k_values=TOP_K_VALUES,
            corpus=corpus
        )
    pd.DataFrame.from_dict(gs_results, orient='index').reset_index().rename(columns={'index': 'name'}).to_csv(f"../output/{corpus}/gs_results.tsv", sep='\t', index=False)
    pd.DataFrame.from_dict(clean_results, orient='index').reset_index().rename(columns={'index': 'name'}).to_csv(f"../output/{corpus}/clean_results.tsv", sep='\t', index=False)
    pd.DataFrame.from_dict(um_results, orient='index').reset_index().rename(columns={'index': 'name'}).to_csv(f"../output/{corpus}/um_results.tsv", sep='\t', index=False)
    pd.DataFrame.from_dict(uc_results, orient='index').reset_index().rename(columns={'index': 'name'}).to_csv(f"../output/{corpus}/uc_results.tsv", sep='\t', index=False)



2025-04-07 08:29:47,037 - INFO - Vocabulary initialized successfully. (faissEncoder.py:76)
2025-04-07 08:30:59,571 - INFO - FAISS index fitted successfully. (faissEncoder.py:154)
2025-04-07 08:31:21,732 - INFO - Calculating Top-k accuracy... (metrics.py:39)
2025-04-07 08:31:21,732 - INFO - DataFrame size: 2598, Top-k values: [1, 5, 25, 50, 100, 200] (metrics.py:40)
2025-04-07 08:31:21,900 - INFO - Top-1 accuracy: 0.5855 (metrics.py:67)
2025-04-07 08:31:21,901 - INFO - Top-5 accuracy: 0.7244 (metrics.py:67)
2025-04-07 08:31:21,901 - INFO - Top-25 accuracy: 0.8114 (metrics.py:67)
2025-04-07 08:31:21,901 - INFO - Top-50 accuracy: 0.8418 (metrics.py:67)
2025-04-07 08:31:21,902 - INFO - Top-100 accuracy: 0.8622 (metrics.py:67)
2025-04-07 08:31:21,902 - INFO - Top-200 accuracy: 0.8737 (metrics.py:67)
2025-04-07 08:31:21,903 - INFO - Top-k accuracy calculation complete. (metrics.py:69)
2025-04-07 08:31:21,903 - INFO - Calculating Top-k accuracy... (metrics.py:39)
2025-04-07 08:31:21,903 - INF

Reranking candidates: 100%|██████████| 2598/2598 [04:35<00:00,  9.44it/s]

2025-04-07 08:36:39,820 - INFO - Reranking complete. (crossEncoder.py:205)
2025-04-07 08:36:39,821 - INFO - Calculating Top-k accuracy... (metrics.py:39)
2025-04-07 08:36:39,821 - INFO - DataFrame size: 2598, Top-k values: [1, 5, 25, 50, 100, 200] (metrics.py:40)


2025-04-07 08:36:39,991 - INFO - Top-1 accuracy: 0.5939 (metrics.py:67)
2025-04-07 08:36:39,992 - INFO - Top-5 accuracy: 0.7386 (metrics.py:67)
2025-04-07 08:36:39,992 - INFO - Top-25 accuracy: 0.8345 (metrics.py:67)
2025-04-07 08:36:39,992 - INFO - Top-50 accuracy: 0.8610 (metrics.py:67)
2025-04-07 08:36:39,993 - INFO - Top-100 accuracy: 0.8703 (metrics.py:67)
2025-04-07 08:36:39,993 - INFO - Top-200 accuracy: 0.8737 (metrics.py:67)
2025-04-07 08:36:39,993 - INFO - Top-k accuracy calculation complete. (metrics.py:69)
2025-04-07 08:36:40,000 - INFO - Reranking candidates... (crossEncoder.py:193)


Reranking candidates: 100%|██████████| 2507/2507 [04:36<00:00,  9.08it/s]

...

Reranking candidates: 100%|██████████| 763/763 [01:16<00:00,  9.91it/s]

2025-04-07 11:56:06,691 - INFO - Reranking complete. (crossEncoder.py:205)
2025-04-07 11:56:06,692 - INFO - Calculating Top-k accuracy... (metrics.py:39)
2025-04-07 11:56:06,692 - INFO - DataFrame size: 763, Top-k values: [1, 5, 25, 50, 100, 200] (metrics.py:40)


2025-04-07 11:56:06,744 - INFO - Top-1 accuracy: 0.3408 (metrics.py:67)
2025-04-07 11:56:06,745 - INFO - Top-5 accuracy: 0.5675 (metrics.py:67)
2025-04-07 11:56:06,745 - INFO - Top-25 accuracy: 0.7575 (metrics.py:67)
2025-04-07 11:56:06,746 - INFO - Top-50 accuracy: 0.7916 (metrics.py:67)
2025-04-07 11:56:06,746 - INFO - Top-100 accuracy: 0.8113 (metrics.py:67)
2025-04-07 11:56:06,747 - INFO - Top-200 accuracy: 0.8139 (metrics.py:67)
2025-04-07 11:56:06,747 - INFO - Top-k accuracy calculation complete. (metrics.py:69)
2025-04-07 11:56:12,141 - INFO - Vocabulary initialized successfully. (faissEncoder.py:76)
2025-04-07 11:57:39,388 - INFO - FAISS index fitted successfully. (faissEncoder.py:154)
2025-04-07 11:58:03,905 - INFO - Calculating Top-k accuracy... (metrics.py:39)
2025-04-07 11:58:03,906 - INFO - DataFrame size: 2848, Top-k values: [1, 5, 25, 50, 100, 200] (metrics.py:40)
2025-04-07 11:58:04,071 - INFO - Top-1 accuracy: 0.6257 (metrics.py:67)
2025-04-07 11:58:04,071 - INFO - Top

Some weights of RobertaModel were not initialized from the model checkpoint at PlanTL-GOB-ES/roberta-base-biomedical-clinical-es and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


2025-04-07 12:07:22,608 - INFO - Vocabulary initialized successfully. (faissEncoder.py:76)
2025-04-07 12:08:45,721 - INFO - FAISS index fitted successfully. (faissEncoder.py:154)
2025-04-07 12:09:10,021 - INFO - Calculating Top-k accuracy... (metrics.py:39)
2025-04-07 12:09:10,022 - INFO - DataFrame size: 2848, Top-k values: [1, 5, 25, 50, 100, 200] (metrics.py:40)
2025-04-07 12:09:10,189 - INFO - Top-1 accuracy: 0.4210 (metrics.py:67)
2025-04-07 12:09:10,190 - INFO - Top-5 accuracy: 0.5137 (metrics.py:67)
2025-04-07 12:09:10,190 - INFO - Top-25 accuracy: 0.6004 (metrics.py:67)
2025-04-07 12:09:10,191 - INFO - Top-50 accuracy: 0.6327 (metrics.py:67)
2025-04-07 12:09:10,191 - INFO - Top-100 accuracy: 0.6671 (metrics.py:67)
2025-04-07 12:09:10,191 - INFO - Top-200 accuracy: 0.6857 (metrics.py:67)
2025-04-07 12:09:10,192 - INFO - Top-k accuracy calculation complete. (metrics.py:69)
2025-04-07 12:09:10,192 - INFO - Calculating Top-k accuracy... (metrics.py:39)
2025-04-07 12:09:10,192 - INF